# Survey Evaluation

## Loading packages

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Dictionaries with the right answers

In [21]:
# Yes-no questions: simple check
# Problem-solving: manual?
# Cloze test: automatic + manual?

def construct_yes_no_correct():
    answers = dict()

    with open("answers/VB_yesno.txt") as ansfile:
        ans = ansfile.readlines()
        ans = [a.strip() for a in ans]
        answers["VB"] = ans

    with open("answers/FE_yesno.txt") as ansfile:
        ans = ansfile.readlines()
        ans = [a.strip() for a in ans]
        answers["FE"] = ans

    return answers

def construct_cloze_correct():
    answers = {"VB": [], "FE": []}

    with open("answers/VB_cloze.txt") as ansfile:
        VB_answers = ansfile.readlines()
        for q in VB_answers:
            ans = q.split(",")
            ans = [a.strip() for a in ans]
            answers["VB"].append(ans)

    with open("answers/FE_cloze.txt") as ansfile:
        FE_answers = ansfile.readlines()
        for q in FE_answers:
            ans = q.split(",")
            ans = [a.strip() for a in ans]
            answers["FE"].append(ans)

    return answers

def construct_correct():
    yes_no = construct_yes_no_correct()
    cloze = construct_cloze_correct()
    return yes_no, cloze

## Loading file

In [22]:
def load_data(data_location, n_subjects):
    survey_data = pd.read_excel(data_location)
    subjects_data = survey_data.iloc[-n_subjects:]
    return subjects_data

n_subjects = 20
data_location = "~/Projects/Cognitive Workload/survey_data.xlsx"

## Extracting columns

In [23]:
def extract_answers(data):
    yes_no_answers = dict()
    yes_no_answers["VB"] = data.loc[:, "VB_C1":"VB_C11"]
    yes_no_answers["FE"] = data.loc[:, "FAC_1":"FAC_11"]

    problem_solving_answers = dict()
    problem_solving_answers["VB"] = data.loc[:, "VB_P1":"VB_P5"]
    problem_solving_answers["FE"] = data.loc[:, "FE_P1":"FE_P5"]

    cloze_answers = dict()
    cloze_answers["VB"] = data["Q86"]
    cloze_answers["FE"] = data["Q88"]

    return yes_no_answers, cloze_answers

## Checking answers

In [24]:
def check_yes_no(correct, answers):
    score = 0
    total = len(correct)
    for i in range(len(correct)):
        if answers[i].lower() == correct[i].lower():
            score += 1
    return score

def check_cloze(correct, answers):
    score = 0
    for i in range(len(correct)):
        if answers[i].lower() in correct[i]:
            score += 1
        else:
            # Ask whether it is a synonym
            continue
    return score

## Storage of data

In [25]:
control_VB = np.zeros((n_subjects, 3))
control_FE = np.zeros((n_subjects, 3))
treatment_VB = np.zeros((n_subjects, 3))
treatment_FE = np.zeros((n_subjects, 3))

## Evaluation

In [26]:

data = load_data(data_location, n_subjects)

yes_no_correct, cloze_correct = construct_correct()
yes_no_answers, cloze_answers = extract_answers(data)

# Determine whether participant is control or treatment
# Calculate scores and put in matrix

/home/andreas/.virtualenv/university/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
